This code takes in a sentence and parses it to the correct grammatical form then plays the correct videos needed

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import wordnet as wn
import string
import spacy
import itertools
import os

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nlp = spacy.load('en_core_web_sm')
# nlp = spacy.load('en_core_web_lg')  #takess longer to load but is a larger dictionary 

In [ ]:
timeOrderWords = ['aurora', 'afternoon','bedtime', 'break of day', 'break of the day', 'breakfast time','canonical hour','closing time',
 'cockcrow','complin','compline','crepuscle','crepuscule','dawn','dawning','daybreak','dayspring','dinnertime',
 'dusk','early morning hour','evenfall','evening','evensong','fall','first light','gloam','gloaming','happy hour','high noon','late night hour',
 'lights out','lunch period','lunchtime','matins','mealtime','midday','midnight','morning','morning prayer','night','nightfall','none','nones','noon',
 'noonday','noontide','prime','rush hour','sext','small hours','sundown','sunrise','sunset','sunup','suppertime','terce','tierce','twelve noon','twilight','vespers','zero hour']

In [ ]:
sentences=["The big black cat stared at the small dog.",
           "I won't watch her brother in the evenings.",
           "Which car did Jane buy?"]


In [ ]:
def translateToASL(sentence:list) -> list:
  doc = nlp(sentence)
  # for word in doc
  tokens_list = []
  signs_list = []
  for token in doc:
    # remove auxilaries, punctuation, determiners, prepositions
      if token.pos_ not in ['AUX', 'PUNCT'] and token.tag_ not in ['DT','IN']:
          tokens_list.append([token.text, token.lemma_, token.pos_, token.tag_,])
          signs_list.append(token.lemma_)

  for sign in signs_list:
    signs_list[signs_list.index(sign)] = sign.lower()
  # print(signs_list)

  if 'not' in signs_list: 
    ind = signs_list.index('not')
    signs_list[ind], signs_list[ind+1] = signs_list[ind+1], signs_list[ind]
  # not comes after the verb it negates in ASL
  # print(signs_list)

  if 'i' in signs_list:
    ind = signs_list.index('i')
    signs_list[ind] = 'me'
  # 'me' is used instead of 'i' in ASL
  # print(signs_list)

  for sign in signs_list:
    if sign in timeOrderWords:
      word = signs_list[signs_list.index(sign)]
      signs_list.remove(word)
      signs_list.insert(0, word)
  # time order words need to be at the start of the sentence in ASL
  return signs_list

In [ ]:
for x in sentences: print(translateToASL(x)) 

['big', 'black', 'cat', 'stare', 'small', 'dog']
['evening', 'me', 'watch', 'not', 'her', 'brother']
['which', 'car', 'jane', 'buy']


In [ ]:
# timeOrderWords = [synset.lemma_names() for synset in wn.synsets('time_of_day')[0].hyponyms()]
# for x in wn.synsets('time_of_day')[0].hyponyms():
#   if len(x.hyponyms()) > 0:
#     timeOrderwords = [timeOrderWords.append(y.lemma_names()) for y in x.hyponyms()]
# timeOrderWords = list(itertools.chain(*timeOrderWords))
# timeOrderWords = [timeOrderWords[x].replace('_', " ") for x in range(len(timeOrderWords))]
# timeOrderWords = [timeOrderWords[x].replace('-', " ") for x in range(len(timeOrderWords))]
# timeOrderWords

In [ ]:
directory = ""

In [ ]:
# if word not found in videos, display letter by letter spelling
def findfiles(directory:str, sentence:str) -> list:
  sentence = translateToASL(sentence)
  videos = list(os.listdir(directory))
  for x in sentence:
    if x+".mp4" not in videos:
      temp = list(x)
      sentence.insert(sentence.index(x), temp)
      sentence.remove(x)

  sentence = list(itertools.chain(*sentence))  #flatten list
  sentence = [x+".mp4" for x in sentence] # add video name extension
  return sentence